Найдите на ютубе интересную лекцию и запустите на ней нейро-копирайтера. Получите аудио-дорожку, на шаге транскрибации переведите текст на английский язык (для этого изучите документацию whisper - https://github.com/openai/whisper).

Сделайте нейро консультанта на базе, который бы отвечал на русском языке, а методичку - на английском.

In [ ]:
yt_urls = ['https://youtu.be/Bu8RfKEo6ww']
YouTube_video_title = "cold sales"

In [ ]:
!pip install -q git+https://github.com/ytdl-org/youtube-dl.git
!pip install -q git+https://github.com/openai/whisper.git

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q tiktoken==0.4.0 openai==0.28.0 langchain==0.0.281 faiss-cpu==1.7.4

In [ ]:
!pip install -qq python-docx
!pip install -U pytube
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import re
import time
from IPython.display import HTML, clear_output
import subprocess
from pathlib import Path
import json
import ipywidgets as widgets
from IPython.display import display

from tqdm.auto import tqdm
import getpass
import pickle
#import torch
from pytube import YouTube
import tiktoken
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import whisper
import openai

# Сохраняем во временное хранилище
import codecs
from langchain.chains import ConversationChain         # Импортируем класс для создания цепочек диалогов
from langchain.chat_models import ChatOpenAI           # Импортируем класс для работы с чатами на базе OpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory  # Импортируем класс для управления памятью диалогов
from langchain.text_splitter import MarkdownHeaderTextSplitter, Document, RecursiveCharacterTextSplitter
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
# Получаем первую (и единственную) ссылку из списка yt_urls
url = yt_urls[0]

# Задаем путь и формат названия файла
output_template = f'tmp/{YouTube_video_title}.m4a'

# Загружаем аудио с лучшим качеством (в формате m4a) и сохраняем его под определенным именем
!youtube-dl $url -f 'bestaudio[ext=m4a]' -o "$output_template"

# Указываем путь к файлу, который будем обрабатывать
mp4_file = output_template

[youtube] Bu8RfKEo6ww: Downloading webpage
[youtube] Bu8RfKEo6ww: Downloading MPD manifest
[youtube] Bu8RfKEo6ww: Downloading player da154528
[dashsegments] Total fragments: 14
[download] Destination: tmp/cold sales.m4a
[download] 100% of 134.23MiB in 00:30
[ffmpeg] Correcting container in "tmp/cold sales.m4a"


In [ ]:
# Используем Whisper для обработки аудиофайла
!whisper "$output_template" --model large --language Russian --task translate

100%|█████████████████████████████████████| 2.88G/2.88G [01:12<00:00, 42.8MiB/s]
[00:00.000 --> 00:07.000]  Today there will be a webinar about how to get into a cold or through semi-cold contacts in the company
[00:07.000 --> 00:10.000]  And about the funnel, what is it, why is it important, and so on
[00:10.000 --> 00:14.000]  And on the next 28th there will be a detailed funnel of questions
[00:14.000 --> 00:19.000]  Be super careful, because this is the information that you will not find anywhere
[00:19.000 --> 00:22.000]  It is very difficult to get quality information
[00:22.000 --> 00:28.000]  There is a book on sales, but it is difficult to study
[00:28.000 --> 00:31.000]  And it's just a huge book, you have to read it for a long time
[00:31.000 --> 00:34.000]  There is a lot of things on the Internet, but not very proven
[00:34.000 --> 00:38.000]  I taught our sales department, and Evgeniy himself studied
[00:38.000 --> 00:41.000]  That is, it is already proven
[00:41.000 --> 

In [ ]:
def get_key_ОpenAI():
  openai.api_key = getpass.getpass(prompt='Введите секретный ключ для сервиса chatGPT: ')
  os.environ["OPENAI_API_KEY"] = openai.api_key

get_key_ОpenAI()

Введите секретный ключ для сервиса chatGPT: ··········


In [ ]:
#@title Напишите самостоятельно промпт в систем и юзер для разделения текста на логические блоки с выделением названия раздела:
system = 'You are a genius of text, copywriting, writing. Your task is to recognise fragments in the text and break it down into these fragments, preserving the integrity of the text 100%. ' #@param {type:"string"}
user = "Please let's think step by step: Think about which sections in the text you can recognise and what meaningful title you can give to each section. Next, write your answer across the previous answer in order: ## Title of section, followed by all text related to that section. Text:" #@param {type:"string"}

temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}
chunk_size = 6000 #@param {type: "slider", min: 1000, max: 7000, step:500}

In [ ]:
# @title Функции
# Функция настройки стиля для переноса текста в выводе ячеек
# для изменения стиля отображения текста, так чтобы предотвратить переполнение текста за границы ячейки вывода и обеспечить его перенос.
def set_text_wrap_css():
    css = '''
    <style>
    pre {
        white-space: pre-wrap;
    }
    </style>
    '''
    display(HTML(css))

get_ipython().events.register('pre_run_cell', set_text_wrap_css)

# Функция подсчета количества токенов
def num_tokens_from_messages(messages, model='gpt-3.5-turbo-0301'):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding('cl100k_base')

    if model in ['gpt-3.5-turbo-0301', 'gpt-3.5-turbo-0613', 'gpt-3.5-turbo-16k', 'gpt-3.5-turbo', 'gpt-3.5-turbo-1106']:
        num_tokens = 0

        for message in messages:
            num_tokens += 4

            for key, value in message.items():
                num_tokens += len(encoding.encode(value))

                if key == 'name':
                    num_tokens -= 1

        num_tokens += 2
        return num_tokens

    else:
        raise NotImplementedError(f'''num_tokens_from_messages() is not presently implemented for model {model}.''')


# Функция дробления текста на чанки
def split_text(txt_file, chunk_size=chunk_size):
    source_chunks = []
    splitter = RecursiveCharacterTextSplitter(separators=['\n', '\n\n', '. '], chunk_size=chunk_size, chunk_overlap=0)

    for chunk in splitter.split_text(txt_file):
        source_chunks.append(Document(page_content=chunk, metadata={}))

    print(f'\n\nТекст разбит на {len(source_chunks)} чанков.')

    return source_chunks


# Функция получения ответа от модели
def answer_index(system, user, chunk, temp=temperature, model='gpt-3.5-turbo'):

    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user + f'{chunk}'}
    ]

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    # Вывод количества токенов отключен
    # print(f'\n====================\n\n{num_tokens_from_messages(messages)} токенов будет использовано на чанк\n\n')
    answer = completion.choices[0].message.content

    return answer


def process_one_file(file_path, system, user):
    with open(file_path, 'r') as txt_file:
        text = txt_file.read()
    source_chunks = split_text(text)
    processed_text = ''
    unprocessed_text = ''

    for chunk in source_chunks:
        attempt = 0

        while attempt < 3:
            try:
                answer = answer_index(system, user, chunk.page_content)
                break  # Успешно получили ответ, выходим из цикла попыток

            except Exception as e:
                attempt += 1  # Увеличиваем счетчик попыток
                print(f'\n\nПопытка {attempt} не удалась из-за ошибки: {str(e)}')
                time.sleep(10)  # Ожидаем перед следующей попыткой
                if attempt == 3:
                    answer = ''
                    print(f'\n\nОбработка элемента {chunk} не удалась после 3 попыток')
                    unprocessed_text += f'{chunk}\n\n'

        processed_text += f'{answer}\n\n'  # Добавляем ответ в результат
        print(f'{answer}')  # Выводим ответ

    return processed_text, unprocessed_text

In [ ]:
# @title Запуск
file_path = f'/content/{YouTube_video_title}.txt'
# Вызываем функцию обработки для этого файла
processed_text, unprocessed_text = process_one_file(file_path, system, user)



Текст разбит на 19 чанков.
## Section 1: Introduction to the webinar
Text: Today there will be a webinar about how to get into a cold or through semi-cold contacts in the company
And about the funnel, what is it, why is it important, and so on
And on the next 28th there will be a detailed funnel of questions
Be super careful, because this is the information that you will not find anywhere
It is very difficult to get quality information
There is a book on sales, but it is difficult to study
And it's just a huge book, you have to read it for a long time
There is a lot of things on the Internet, but not very proven
I taught our sales department, and Evgeniy himself studied
That is, it is already proven
Second, it will not be so much related to artificial intelligence
That is, take it as a separate webinar on sales
Yes, project it for yourself on artificial intelligence
I will be with you in the chat to see if there will be questions
How is it interesting?
If you want to integrate it wit

In [ ]:
# теперь в переменной
processed_text

"## Section 1: Introduction to the webinar\nText: Today there will be a webinar about how to get into a cold or through semi-cold contacts in the company\nAnd about the funnel, what is it, why is it important, and so on\nAnd on the next 28th there will be a detailed funnel of questions\nBe super careful, because this is the information that you will not find anywhere\nIt is very difficult to get quality information\nThere is a book on sales, but it is difficult to study\nAnd it's just a huge book, you have to read it for a long time\nThere is a lot of things on the Internet, but not very proven\nI taught our sales department, and Evgeniy himself studied\nThat is, it is already proven\nSecond, it will not be so much related to artificial intelligence\nThat is, take it as a separate webinar on sales\nYes, project it for yourself on artificial intelligence\nI will be with you in the chat to see if there will be questions\nHow is it interesting?\nIf you want to integrate it with neurons, w

Итак, мы получили текст транскрибации, разделенный на разделы с названиями данных разделов. Теперь разделим этот текст на чанки при помощи MarkdownHeaderTextSplitter, создадим из него векторную базу Faiss и сделаем нейро-консультанта, который отвечает на вопросы по тексту на русском языке, а из текста транскрибации с разделами составим методичку на английском.

# Создаем нейро-консультанта:

In [ ]:
# Определяем заголовки, на которые следует разбить текст
headers_to_split_on = [
    ("##", "Header 2")
    ]
# Создаем объект для разбиения текста на секции по заголовкам
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Получаем список документов, разбитых по заголовкам
md_header_splits = markdown_splitter.split_text(processed_text)

In [ ]:
md_header_splits

[Document(page_content="Text: Today there will be a webinar about how to get into a cold or through semi-cold contacts in the company\nAnd about the funnel, what is it, why is it important, and so on\nAnd on the next 28th there will be a detailed funnel of questions\nBe super careful, because this is the information that you will not find anywhere\nIt is very difficult to get quality information\nThere is a book on sales, but it is difficult to study\nAnd it's just a huge book, you have to read it for a long time\nThere is a lot of things on the Internet, but not very proven\nI taught our sales department, and Evgeniy himself studied\nThat is, it is already proven\nSecond, it will not be so much related to artificial intelligence\nThat is, take it as a separate webinar on sales\nYes, project it for yourself on artificial intelligence\nI will be with you in the chat to see if there will be questions\nHow is it interesting?\nIf you want to integrate it with neurons, write in the chat, I 

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(md_header_splits, embeddings)

In [ ]:
system_for_NA = "You are a practical psychologist and business sales trainer. Your task is to answer the student's question only on the basis of the documents presented to you, without adding anything from your own knowledge. Answer the student in Russian."

In [ ]:
def answer_neuro_assist(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=3)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nExcerpt of document №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system_for_NA},
        {"role": "user", "content": f"Answer the student's question. Answer in Russian. Do not mention the excerpts of the student response information document in your answer. Student response document: {message_content}\n\nStudent Question: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [ ]:
topic="Как перестать боятся продавать?"
ans=answer_neuro_assist(system, topic, db, verbose=1)
ans


 ===========================================: 
message_content :
 
Excerpt of document №1
=====================Text: Look, the first thing
A person who does not sell daily
For whom it is not a daily profession
But when he thinks about
That you need to somehow
Your services
Your professional competence
Sell to people
Who are not his customers
who are not waiting for it. The very first collision is with fear, with your own fear.
So, now I'll try how to list it, it's probably better here, it will still work out. Yes.
And please write in the chat, what do you think, what is the main fear of the seller of their services?
Well, your fear, perhaps, as you imagine, when you come across the fact that somehow you need to call the company,
somehow meet, offer.
And today, today, let's say, after our meeting,
after our webinar,
when you will be afraid, and you will be afraid if you are people, and this is absolutely normal,
then what will you be afraid of? This is one question.
And if you have a f

'Чтобы перестать бояться продавать, важно осознать свои страхи и позволить себе быть несовершенным. Главное в бизнесе - не стремиться быть идеальным. Необходимо устанавливать контакты, допускать неудачи и получать отказы. Важно также задуматься о внутреннем послании, с которым вы вступаете в контакт с клиентами. Основная цель не должна быть только продажей, а может быть, например, установлением отношений или предоставлением информации.'

In [ ]:
topic="Как нужно разговариваить с клиентом?"
ans=answer_neuro_assist(system, topic, db, verbose=1)
ans


 ===========================================: 
message_content :
 
Excerpt of document №1
=====================Text: And now, with the permission of Dmitry, I will include a recording.
I asked him to tell me how he sees a conversation with a potential client.
How does he generally propose to talk?
And Dmitry ...
Dmitry dictated some text.
Well, usually I speak.
Usually I teach to speak like this.
And I think that Dmitry has already, perhaps, spoken about this with you or will speak.
But now I will turn on his voice so that you listen to the correspondence with these three technical criteria that I have already written.
Just a second.  
Excerpt of document №2
=====================Text: Well, let's go
We will be with you
First of all, hello
We will work with you for two and a half hours
With a 10-minute break
I will tell you what I know
What I did and do myself
From time to time
Sometimes I will ask you questions
And then I will ask you to answer in the chat
When Dima talked to me about

'Для успешного разговора с клиентом важно принять инициативу в свои руки и задавать ему вопросы, чтобы узнать его потребности и интересы. Необходимо проявлять уверенность и говорить с убеждением, но при этом давать клиенту возможность отказаться, если предложение не подходит ему. Также важно строить диалог таким образом, чтобы клиент чувствовал, что вы не пытаетесь ему что-то продать прямо сейчас, а помогаете ему принять решение.'

Делаем методичку:

In [ ]:
#@title Обрабатываем каждый чанк, выделяя только суть для методички
system = "You are a copywriting genius, an expert in sales and practical psychology. You receive a section of raw text on a certain topic. You need to extract from this text the essence, only the most important, keeping all the necessary details, but removing all the \"water\" and words (sentences) that do not carry a semantic load." #@param {type:"string"}
user = "From this text, select only the information that is valuable in terms of the topic \"Sales\". Remove all the \"water\". As a result, you should have a section for a manual on this topic. Base it only on the text given to you, don't make up anything of your own. The answer should be in the format ## Title of the text, and then the valuable information from the text. If there is no valuable information in the text, then leave only the title of the section, for example: \"## Introduction\". Text:" #@param {type:"string"}

temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}


In [ ]:
def process_documents(documents, system, user, temperature):
    """
    Функция принимает чанки, system, user, temperature для модели.
    Она обрабатывает каждый документ, используя модель GPT, конкатенирует результаты в один текст и сохраняет в файл .txt.
    """
    processed_text_for_handbook = ""  # Строка для конкатенации обработанного текста

    for document in documents:
        # Форматируем метаданные для включения в чанк
        metadata_str = "\n".join([f"{key}: {value}" for key, value in document.metadata.items()])
        # Конкатенируем метаданные и содержание документа для передачи в функцию
        chunk_with_metadata = f"{metadata_str}\n\n{document.page_content}"

        # Получаем ответ от модели
        answer = answer_index(system, user, chunk_with_metadata, temperature, model='gpt-3.5-turbo')
        # Добавляем обработанный текст в общую строку
        processed_text_for_handbook += f"{answer}\n\n"

    # Записываем полученный текст в файл
    with open('processed_documents.txt', 'w', encoding='utf-8') as f:
        f.write(processed_text_for_handbook)

    # Функция возвращает путь к файлу с обработанным текстом
    return 'processed_documents.txt'

# Применение функции
file_path = process_documents(md_header_splits, system, user, temperature)
print(f"Обработанный текст сохранен в файле: {file_path}")

Обработанный текст сохранен в файле: processed_documents.txt


In [ ]:
# Читаем и выведим содержимое методички:
file_path = '/content/processed_documents.txt'

with open(file_path, 'r') as file:
    content = file.read()
    print(content)

## Introduction to the webinar

Today's webinar will cover strategies for successfully approaching cold or semi-cold contacts within a company. We will also discuss the importance of the sales funnel and its role in the sales process. This webinar will provide valuable information that is not easily accessible elsewhere. While there are books and online resources available, they may not offer the same level of quality and proven techniques. Our sales department has already benefited from this training, and it is highly recommended for anyone interested in improving their sales skills. Although there will be some mention of artificial intelligence, this webinar primarily focuses on sales techniques rather than AI integration. Feel free to ask questions in the chat, and if you are interested in integrating sales with AI, let us know. Prepare practical questions for future webinars as we plan to explore the integration of sales with artificial intelligence. This webinar is primarily cente